In [1]:
import os
from nltk.corpus import stopwords
from nltk import word_tokenize
from string import punctuation
import random

import nltk

In [2]:
s = str.maketrans('', '', punctuation)
a = 'salam!@'
a.translate(s)

'salam'

In [3]:
stop_words = stopwords.words('english')

In [4]:
negative_documents = []
max_len_negative = 0
for file in os.listdir('data/neg'):
    with open('data/neg/' + file) as f:
        text = f.read()
        tokens = word_tokenize(text)
        translator = str.maketrans('', '', punctuation)
        tokens = [w.translate(translator) for w in tokens]
        tokens = [w for w in tokens if not w in stop_words]
        if len(tokens) > max_len_negative:
            max_len_negative = len(tokens)
        negative_documents.append(' '.join(tokens))
len(negative_documents)

1000

In [5]:
max_len_negative

1400

In [6]:
positive_documents = []
max_len_positive = 0
for file in os.listdir('data/pos'):
    with open('data/pos/' + file) as f:
        text = f.read()
        tokens = word_tokenize(text)
        translator = str.maketrans('', '', punctuation)
        tokens = [w.translate(translator) for w in tokens]
        tokens = [w for w in tokens if not w in stop_words]
        if len(tokens) > max_len_positive:
            max_len_positive = len(tokens)
        positive_documents.append(' '.join(tokens))
len(positive_documents)

1000

In [7]:
max_len_positive

1693

In [8]:
max_len = max(max_len_negative, max_len_positive)

In [9]:
random.shuffle(negative_documents)
random.shuffle(positive_documents)

In [10]:
X_train = negative_documents[:800] + positive_documents[:800]

In [11]:
len(X_train)

1600

In [12]:
y_train = [0 for _ in range(800)] + [1 for _ in range(800)]

In [13]:
len(y_train)

1600

In [14]:
X_test = negative_documents[800:] + positive_documents[800:]
len(X_test)

400

In [15]:
y_test = [0 for _ in range(200)] + [1 for _ in range(200)]

In [16]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import plot_model
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Flatten, Embedding, Conv1D, MaxPool1D, Dropout
from tensorflow.keras.layers import concatenate

2025-04-09 16:07:16.098059: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-09 16:07:16.107916: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744207636.119555    7554 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744207636.123030    7554 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1744207636.131438    7554 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [17]:
max_len

1693

In [18]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

In [19]:
import pickle
with open('tokenizer.h5', 'wb') as f:
    pickle.dump(tokenizer, f)

In [20]:
tokenizer

In [21]:
vocab_len = len(tokenizer.word_index) + 1

In [22]:
vocab_len

40656

In [23]:
encoded = tokenizer.texts_to_sequences(X_train)

In [24]:
encoded[0]

[28,
 244,
 2885,
 591,
 879,
 5490,
 278,
 98,
 4,
 678,
 294,
 16,
 4040,
 26,
 22,
 1051,
 2589,
 2520,
 2234,
 3,
 18235,
 100,
 1084,
 7898,
 2660,
 11407,
 9243,
 1753,
 244,
 23842,
 196,
 5491,
 253,
 4771,
 23843,
 540,
 1333,
 2051,
 739,
 2,
 129,
 141,
 9,
 159,
 132,
 64,
 879,
 5490,
 98,
 57,
 129,
 90,
 113,
 310,
 261,
 562,
 437,
 3201,
 1725,
 264,
 98,
 2390,
 77,
 133,
 777,
 1726,
 1847,
 4208,
 1,
 547,
 533,
 83,
 332,
 2441,
 186,
 1848,
 2723,
 2590,
 1626,
 28,
 23844,
 48,
 384,
 20,
 29,
 245,
 9,
 3032,
 521,
 351,
 3764,
 879,
 5490,
 1,
 2810,
 23845,
 261,
 2948,
 772,
 36,
 1535,
 15061,
 23846,
 18236,
 278,
 333,
 9,
 4041,
 810,
 9,
 226,
 49,
 4,
 11,
 11,
 3765,
 936,
 596,
 562,
 332,
 172,
 48,
 64,
 2009,
 26,
 5,
 72,
 211,
 1189,
 18237,
 5801,
 228,
 2,
 130,
 59,
 867,
 1754,
 7899,
 844,
 2442,
 2235,
 174,
 64,
 879,
 5490,
 10222,
 562,
 92,
 603,
 48,
 351,
 1,
 1535,
 533,
 2723,
 15,
 139,
 2236,
 27,
 392,
 2391,
 1,
 254,
 879,
 549

In [25]:
padded = pad_sequences(encoded, maxlen=max_len, padding='post')

In [26]:
padded.shape

(1600, 1693)

In [27]:
input1 = Input(shape=(max_len,))
embedding1 = Embedding(vocab_len, 100)(input1)
conv1 = Conv1D(filters=32, kernel_size=4, activation='relu')(embedding1)
drop1 = Dropout(0.5)(conv1)
pool1 = MaxPool1D(pool_size=2)(drop1)
flat1 = Flatten()(pool1)
dense1 = Dense(10, activation='relu')(flat1)
output = Dense(1, activation='sigmoid')(dense1)

2025-04-09 16:07:24.935695: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: CUDA_ERROR_UNKNOWN: unknown error
2025-04-09 16:07:24.935718: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:178] verbose logging is disabled. Rerun with verbose logging (usually --v=1 or --vmodule=cuda_diagnostics=1) to get more diagnostic output from this module
2025-04-09 16:07:24.935722: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:183] retrieving CUDA diagnostic information for host: amirhosein-Legion-5-15ITH6
2025-04-09 16:07:24.935724: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:190] hostname: amirhosein-Legion-5-15ITH6
2025-04-09 16:07:24.935904: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:197] libcuda reported version is: 570.124.6
2025-04-09 16:07:24.935922: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:201] kerne

In [28]:
model = Model(inputs=[input1], outputs=output)

In [29]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [30]:
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 1693)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding (Embedding)           │ (None, 1693, 100)      │     4,065,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d (Conv1D)                 │ (None, 1690, 32)       │        12,832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1690, 32)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ (None, 845, 32)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 27040)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 10)             │       270,410 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            11 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,348,853 (16.59 MB)

 Trainable params: 4,348,853 (16.59 MB)

 Non-trainable params: 0 (0.00 B)

In [31]:
encoded_test = tokenizer.texts_to_sequences(X_test)
padded_test = pad_sequences(encoded_test, maxlen=max_len, padding='post')

In [32]:
padded_test.shape

(400, 1693)

In [33]:
import numpy as np

In [34]:
model.fit([padded], np.array(y_train), epochs=10, batch_size=20, validation_data=([padded_test], np.array(y_test)))

Epoch 1/10
80/80 ━━━━━━━━━━━━━━━━━━━━ 4s 42ms/step - accuracy: 0.4814 - loss: 0.7026 - val_accuracy: 0.5000 - val_loss: 0.6931
Epoch 2/10
80/80 ━━━━━━━━━━━━━━━━━━━━ 3s 40ms/step - accuracy: 0.4760 - loss: 0.6935 - val_accuracy: 0.5000 - val_loss: 0.6931
Epoch 3/10
80/80 ━━━━━━━━━━━━━━━━━━━━ 3s 40ms/step - accuracy: 0.5017 - loss: 0.6930 - val_accuracy: 0.5200 - val_loss: 0.6922
Epoch 4/10
80/80 ━━━━━━━━━━━━━━━━━━━━ 3s 40ms/step - accuracy: 0.5648 - loss: 0.6785 - val_accuracy: 0.6175 - val_loss: 0.6549
Epoch 5/10
80/80 ━━━━━━━━━━━━━━━━━━━━ 3s 40ms/step - accuracy: 0.9178 - loss: 0.3135 - val_accuracy: 0.7975 - val_loss: 0.4431
Epoch 6/10
80/80 ━━━━━━━━━━━━━━━━━━━━ 3s 40ms/step - accuracy: 0.9947 - loss: 0.0241 - val_accuracy: 0.8425 - val_loss: 0.4000
Epoch 7/10
80/80 ━━━━━━━━━━━━━━━━━━━━ 3s 40ms/step - accuracy: 1.0000 - loss: 0.0030 - val_accuracy: 0.8350 - val_loss: 0.4025
Epoch 8/10
80/80 ━━━━━━━━━━━━━━━━━━━━ 3s 40ms/step - accuracy: 1.0000 - loss: 0.0016 - val_accuracy: 0.8350 - v

In [37]:
model.fit([padded], np.array(y_train), epochs=10, batch_size=20, validation_data=([padded_test], np.array(y_test)))

Epoch 1/10
80/80 [==============================] - 7s 93ms/step - loss: 0.6839 - accuracy: 0.5594 - val_loss: 0.6846 - val_accuracy: 0.5325
Epoch 2/10
80/80 [==============================] - 8s 94ms/step - loss: 0.3573 - accuracy: 0.9094 - val_loss: 0.4899 - val_accuracy: 0.7300
Epoch 3/10
80/80 [==============================] - 8s 98ms/step - loss: 0.0303 - accuracy: 0.9981 - val_loss: 0.3883 - val_accuracy: 0.8475
Epoch 4/10
80/80 [==============================] - 8s 94ms/step - loss: 0.0047 - accuracy: 1.0000 - val_loss: 0.3731 - val_accuracy: 0.8350
Epoch 5/10
80/80 [==============================] - 8s 95ms/step - loss: 0.0023 - accuracy: 1.0000 - val_loss: 0.3643 - val_accuracy: 0.8475
Epoch 6/10
80/80 [==============================] - 8s 96ms/step - loss: 0.0014 - accuracy: 1.0000 - val_loss: 0.3639 - val_accuracy: 0.8450
Epoch 7/10
80/80 [==============================] - 7s 91ms/step - loss: 9.9654e-04 - accuracy: 1.0000 - val_loss: 0.3614 - val_accuracy: 0.8450
Epoch 8/1

In [36]:
model.save('textcnn.keras')

In [37]:
plot_model(model, show_shapes=True)

You must install pydot (`pip install pydot`) for `plot_model` to work.


In [38]:
from tensorflow.keras.models import load_model

In [39]:
model = load_model('textcnn.h5')

In [40]:
with open('tokenizer.h5', 'rb') as f:
    tokenizer = pickle.load(f)

In [43]:
tokenizer.word_index

{'film': 1,
 'nt': 2,
 'one': 3,
 'movie': 4,
 'like': 5,
 'even': 6,
 'good': 7,
 'time': 8,
 'would': 9,
 'story': 10,
 'much': 11,
 'also': 12,
 'character': 13,
 'get': 14,
 'characters': 15,
 'two': 16,
 'first': 17,
 'see': 18,
 'well': 19,
 'way': 20,
 'could': 21,
 'make': 22,
 'really': 23,
 'films': 24,
 'life': 25,
 'people': 26,
 'little': 27,
 'plot': 28,
 'never': 29,
 'bad': 30,
 'man': 31,
 'scene': 32,
 'new': 33,
 'best': 34,
 'many': 35,
 'scenes': 36,
 'know': 37,
 'movies': 38,
 'great': 39,
 'another': 40,
 'director': 41,
 'action': 42,
 'love': 43,
 'something': 44,
 'go': 45,
 'end': 46,
 'seems': 47,
 'us': 48,
 'made': 49,
 'back': 50,
 'still': 51,
 'work': 52,
 'makes': 53,
 'world': 54,
 'however': 55,
 'big': 56,
 'every': 57,
 'better': 58,
 'though': 59,
 'take': 60,
 'audience': 61,
 'performance': 62,
 'enough': 63,
 'real': 64,
 'around': 65,
 'last': 66,
 'gets': 67,
 'seen': 68,
 'may': 69,
 'going': 70,
 'role': 71,
 'actually': 72,
 'years': 73,


In [44]:
model.predict()

TypeError: TensorFlowTrainer.predict() missing 1 required positional argument: 'x'